## Were steady states caused by initial condition bias towards 1?

Dynamical system in the thinning paper
Node saturation
Spectral design
Topology paper

In [1]:
%run "../HyperParameterOpt/GenerateExperiments/res_experiment.py"
import copy
from matplotlib import pyplot as plt
import scipy as sp
from scipy import signal
DIFF_EQ_PARAMS = {
                  "x0": [-20, 10, -.5],
                  "begin": 0,
                  "end": 85,
                  "timesteps": 85000,
                  "train_per": .889,
                  "solver": lorenz_equ,
                  "clip": 40
                 }

RES_PARAMS = {
              "uniform_weights": True,
              "solver": "ridge",
              "ridge_alpha": 1e-6,
              "signal_dim": 3,
              "network": "random graph",

              "res_sz": 2500,
              "activ_f": np.tanh,
              "connect_p": .1,
              "spect_rad": 5, #0.5,
              "gamma": 5,
              "sigma": .14,
              "sparse_res": True,
             }

TOL = 5
plt.rcParams["figure.figsize"] = [10, 5]

In [2]:
def adj_matrix(topology, topo_params, n, remove_p=0):
    adj = generate_adj(topology, topo_params, n)
    adj = remove_edges(adj, floor(remove_p*np.sum(adj != 0)))
    return adj

def steady_state_ratio(topo, topo_p, remove_p=0.0):
    DIFF_EQ_PARAMS["x0"] = random_lorenz_x0()
    train_t, test_t, u = rc_solve_ode(DIFF_EQ_PARAMS)
    A = adj_matrix(topo, topo_p, 2500, remove_p=remove_p)
    rc = ResComp(A, **RES_PARAMS)
    rc.state_0 = 2*np.random.rand(rc.res_sz) - 1
    r0 = rc.state_0
    states = rc.drive(train_t, u)
    rank = np.linalg.matrix_rank(states)
    mask = np.abs(np.mean(states, axis=0)) > .97
    rc.state_0 = r0
    error = rc.fit(train_t, u) # Some special kind of L2 error
    n = sum(mask)
    if n == 0:
        mean_ss = np.nan
    else:
        mean_ss = np.mean(np.abs(rc.W_out[:, mask]))
    mean_os = np.mean(np.abs(rc.W_out[:, ~mask]))
    
    return np.round(mean_ss, 4), np.round(mean_os,4), n, error, rank

def rep_ss_ratio(topo, topo_p=None, remove_p=0.0, N=10):
    totss = 0
    totss_weight = 0
    totos_weight = 0
    toterr = 0
    totrank = 0
    for i in range(N):
        ss, os, n, error, rank = steady_state_ratio(topo, topo_p, remove_p=remove_p)
        totss += n
        totss_weight+= ss
        totos_weight += os
        toterr += error
        totrank += rank
        print(f"Trial {i}: {n} Steady states. Avg weight={ss} Oscil. w avg={os} Ratio={np.round(ss/os, 4)} Error={np.round(error, 4)}, Rank={rank}")
    perss = np.round(totss/(N*RES_PARAMS["res_sz"]), 4)
    avgss_weight = np.round(totss_weight/N, 4)
    avgos_weight = np.round(totos_weight/N, 4)
    ratio = np.round(totss_weight/totos_weight, 4)
    avgerr = np.round(toterr/N, 6)
    avgrank = np.round(totrank/N, 4)
    print(f"Average: %ss={perss} ss_weight={avgss_weight} os_weight={avgos_weight} ratio={ratio}, err={avgerr}, rank={avgrank}")

## Watts (Remove 0%)

In [3]:
rep_ss_ratio("watts4", topo_p=0.1, remove_p=0.0)

Trial 0: 1758 Steady states. Avg weight=1.5325 Oscil. w avg=7.3473 Ratio=0.2086 Error=0.006, Rank=1152
Trial 1: 1815 Steady states. Avg weight=1.7862 Oscil. w avg=7.5609 Ratio=0.2362 Error=0.0065, Rank=1087
Trial 2: 1740 Steady states. Avg weight=1.5509 Oscil. w avg=7.7617 Ratio=0.1998 Error=0.0067, Rank=1116
Trial 3: 1814 Steady states. Avg weight=1.7506 Oscil. w avg=9.3105 Ratio=0.188 Error=0.0066, Rank=1090
Trial 4: 1786 Steady states. Avg weight=1.8937 Oscil. w avg=8.0908 Ratio=0.2341 Error=0.0066, Rank=1117
Trial 5: 1852 Steady states. Avg weight=1.7562 Oscil. w avg=9.5162 Ratio=0.1845 Error=0.0076, Rank=1031
Trial 6: 1791 Steady states. Avg weight=1.9121 Oscil. w avg=8.1575 Ratio=0.2344 Error=0.0062, Rank=1135
Trial 7: 1769 Steady states. Avg weight=1.5492 Oscil. w avg=8.5792 Ratio=0.1806 Error=0.0061, Rank=1098
Trial 8: 1813 Steady states. Avg weight=1.7763 Oscil. w avg=8.3123 Ratio=0.2137 Error=0.0062, Rank=1095
Trial 9: 1841 Steady states. Avg weight=1.6533 Oscil. w avg=8.1229

## Watts (Remove 90%)

In [4]:
rep_ss_ratio("watts4", topo_p=0.1, remove_p=0.9)

Trial 0: 303 Steady states. Avg weight=3.1293 Oscil. w avg=3.7674 Ratio=0.8306 Error=0.006, Rank=979
Trial 1: 265 Steady states. Avg weight=2.8409 Oscil. w avg=3.5077 Ratio=0.8099 Error=0.0054, Rank=955
Trial 2: 316 Steady states. Avg weight=2.6206 Oscil. w avg=3.788 Ratio=0.6918 Error=0.0058, Rank=947
Trial 3: 275 Steady states. Avg weight=3.7474 Oscil. w avg=3.8884 Ratio=0.9637 Error=0.0063, Rank=973
Trial 4: 462 Steady states. Avg weight=3.6748 Oscil. w avg=3.9961 Ratio=0.9196 Error=0.0067, Rank=885
Trial 5: 272 Steady states. Avg weight=3.0143 Oscil. w avg=3.4336 Ratio=0.8779 Error=0.0058, Rank=965
Trial 6: 341 Steady states. Avg weight=3.2675 Oscil. w avg=3.7365 Ratio=0.8745 Error=0.0056, Rank=952
Trial 7: 311 Steady states. Avg weight=3.0838 Oscil. w avg=3.7442 Ratio=0.8236 Error=0.0059, Rank=938
Trial 8: 321 Steady states. Avg weight=3.3593 Oscil. w avg=3.9539 Ratio=0.8496 Error=0.0062, Rank=968
Trial 9: 323 Steady states. Avg weight=3.2251 Oscil. w avg=3.7001 Ratio=0.8716 Error

## Geometric (Remove 0%)

In [5]:
rep_ss_ratio("geom", topo_p=4, remove_p=0.0)

Trial 0: 992 Steady states. Avg weight=1.0288 Oscil. w avg=5.3396 Ratio=0.1927 Error=0.0047, Rank=1324
Trial 1: 878 Steady states. Avg weight=0.9288 Oscil. w avg=4.9045 Ratio=0.1894 Error=0.0045, Rank=1362
Trial 2: 533 Steady states. Avg weight=1.1646 Oscil. w avg=4.6271 Ratio=0.2517 Error=0.0044, Rank=1441
Trial 3: 810 Steady states. Avg weight=1.0735 Oscil. w avg=5.0956 Ratio=0.2107 Error=0.0047, Rank=1369
Trial 4: 539 Steady states. Avg weight=0.7515 Oscil. w avg=3.9915 Ratio=0.1883 Error=0.0041, Rank=1450
Trial 5: 786 Steady states. Avg weight=0.8531 Oscil. w avg=4.8343 Ratio=0.1765 Error=0.0047, Rank=1358
Trial 6: 748 Steady states. Avg weight=0.9608 Oscil. w avg=5.002 Ratio=0.1921 Error=0.0047, Rank=1389
Trial 7: 879 Steady states. Avg weight=0.8368 Oscil. w avg=5.2285 Ratio=0.16 Error=0.0048, Rank=1344
Trial 8: 872 Steady states. Avg weight=0.8637 Oscil. w avg=4.784 Ratio=0.1805 Error=0.0047, Rank=1375
Trial 9: 910 Steady states. Avg weight=0.8703 Oscil. w avg=4.7234 Ratio=0.184

## Geometric (Remove 90%)

In [6]:
rep_ss_ratio("geom", topo_p=4.0, remove_p=0.9)

Trial 0: 276 Steady states. Avg weight=3.263 Oscil. w avg=4.0261 Ratio=0.8105 Error=0.0061, Rank=913
Trial 1: 308 Steady states. Avg weight=3.529 Oscil. w avg=3.6268 Ratio=0.973 Error=0.006, Rank=931
Trial 2: 315 Steady states. Avg weight=2.665 Oscil. w avg=3.4759 Ratio=0.7667 Error=0.0059, Rank=898
Trial 3: 305 Steady states. Avg weight=3.5654 Oscil. w avg=3.6106 Ratio=0.9875 Error=0.0059, Rank=921
Trial 4: 240 Steady states. Avg weight=3.846 Oscil. w avg=3.5698 Ratio=1.0774 Error=0.0056, Rank=960
Trial 5: 281 Steady states. Avg weight=2.9224 Oscil. w avg=3.5118 Ratio=0.8322 Error=0.0056, Rank=942
Trial 6: 340 Steady states. Avg weight=2.9648 Oscil. w avg=3.763 Ratio=0.7879 Error=0.0056, Rank=901
Trial 7: 244 Steady states. Avg weight=3.5071 Oscil. w avg=3.4886 Ratio=1.0053 Error=0.0056, Rank=943
Trial 8: 219 Steady states. Avg weight=3.1619 Oscil. w avg=3.7072 Ratio=0.8529 Error=0.0058, Rank=964
Trial 9: 323 Steady states. Avg weight=3.3026 Oscil. w avg=4.0379 Ratio=0.8179 Error=0.00

## Identity (Remove 0%)

In [7]:
rep_ss_ratio("ident", topo_p=None, remove_p=0.0)

Trial 0: 2468 Steady states. Avg weight=302.1844 Oscil. w avg=98.5734 Ratio=3.0656 Error=0.1027, Rank=276
Trial 1: 2481 Steady states. Avg weight=306.9337 Oscil. w avg=63.8406 Ratio=4.8078 Error=0.1084, Rank=264
Trial 2: 2487 Steady states. Avg weight=318.6915 Oscil. w avg=6.4064 Ratio=49.7458 Error=0.1138, Rank=268
Trial 3: 2475 Steady states. Avg weight=303.1277 Oscil. w avg=131.2068 Ratio=2.3103 Error=0.1039, Rank=267
Trial 4: 2486 Steady states. Avg weight=309.3214 Oscil. w avg=5.6176 Ratio=55.0629 Error=0.1098, Rank=248
Trial 5: 2491 Steady states. Avg weight=303.0499 Oscil. w avg=11.7798 Ratio=25.7262 Error=0.1061, Rank=270
Trial 6: 2484 Steady states. Avg weight=300.257 Oscil. w avg=4.3353 Ratio=69.2586 Error=0.1034, Rank=245
Trial 7: 2477 Steady states. Avg weight=296.9908 Oscil. w avg=116.3211 Ratio=2.5532 Error=0.1072, Rank=276
Trial 8: 2487 Steady states. Avg weight=306.451 Oscil. w avg=5.3189 Ratio=57.6155 Error=0.1092, Rank=245
Trial 9: 2477 Steady states. Avg weight=313.1

## Identity (Remove 90%)

In [8]:
rep_ss_ratio("ident", topo_p=None, remove_p=0.9)

Trial 0: 249 Steady states. Avg weight=6.2368 Oscil. w avg=8.9424 Ratio=0.6974 Error=0.0163, Rank=397
Trial 1: 248 Steady states. Avg weight=6.1286 Oscil. w avg=9.2421 Ratio=0.6631 Error=0.0164, Rank=382
Trial 2: 249 Steady states. Avg weight=4.7879 Oscil. w avg=7.2605 Ratio=0.6594 Error=0.0139, Rank=369
Trial 3: 248 Steady states. Avg weight=7.731 Oscil. w avg=9.5316 Ratio=0.8111 Error=0.0171, Rank=398
Trial 4: 250 Steady states. Avg weight=3.6211 Oscil. w avg=6.9261 Ratio=0.5228 Error=0.0144, Rank=372
Trial 5: 249 Steady states. Avg weight=3.9815 Oscil. w avg=8.9926 Ratio=0.4428 Error=0.0171, Rank=381
Trial 6: 248 Steady states. Avg weight=5.3256 Oscil. w avg=8.0233 Ratio=0.6638 Error=0.0153, Rank=368
Trial 7: 247 Steady states. Avg weight=5.286 Oscil. w avg=8.5245 Ratio=0.6201 Error=0.0158, Rank=386
Trial 8: 246 Steady states. Avg weight=5.3715 Oscil. w avg=8.919 Ratio=0.6023 Error=0.0159, Rank=380
Trial 9: 249 Steady states. Avg weight=5.8909 Oscil. w avg=9.0314 Ratio=0.6523 Error=

## Barabasi (Remove 0%)

In [9]:
rep_ss_ratio("barab2", topo_p=None, remove_p=0.0)

Trial 0: 149 Steady states. Avg weight=0.8824 Oscil. w avg=3.6946 Ratio=0.2388 Error=0.0041, Rank=1566
Trial 1: 118 Steady states. Avg weight=1.5468 Oscil. w avg=4.2582 Ratio=0.3633 Error=0.0044, Rank=1716
Trial 2: 96 Steady states. Avg weight=1.3152 Oscil. w avg=4.068 Ratio=0.3233 Error=0.0044, Rank=1662
Trial 3: 135 Steady states. Avg weight=1.2292 Oscil. w avg=4.354 Ratio=0.2823 Error=0.0045, Rank=1685
Trial 4: 81 Steady states. Avg weight=1.3319 Oscil. w avg=4.3031 Ratio=0.3095 Error=0.0044, Rank=1623
Trial 5: 104 Steady states. Avg weight=1.0769 Oscil. w avg=4.0131 Ratio=0.2683 Error=0.0041, Rank=1673
Trial 6: 103 Steady states. Avg weight=1.2102 Oscil. w avg=4.3899 Ratio=0.2757 Error=0.0047, Rank=1617
Trial 7: 163 Steady states. Avg weight=1.2071 Oscil. w avg=3.9273 Ratio=0.3074 Error=0.0043, Rank=1647
Trial 8: 93 Steady states. Avg weight=1.1593 Oscil. w avg=3.9622 Ratio=0.2926 Error=0.0042, Rank=1690
Trial 9: 137 Steady states. Avg weight=1.006 Oscil. w avg=3.9234 Ratio=0.2564 

## Barabasi (Remove 90%)

In [10]:
rep_ss_ratio("barab2", topo_p=None, remove_p=0.9)

Trial 0: 298 Steady states. Avg weight=3.08 Oscil. w avg=3.0723 Ratio=1.0025 Error=0.0054, Rank=880
Trial 1: 226 Steady states. Avg weight=3.4034 Oscil. w avg=3.0424 Ratio=1.1187 Error=0.0053, Rank=903
Trial 2: 204 Steady states. Avg weight=3.1555 Oscil. w avg=3.4333 Ratio=0.9191 Error=0.0058, Rank=889
Trial 3: 292 Steady states. Avg weight=4.01 Oscil. w avg=3.8793 Ratio=1.0337 Error=0.0068, Rank=895
Trial 4: 262 Steady states. Avg weight=2.9891 Oscil. w avg=3.4216 Ratio=0.8736 Error=0.0063, Rank=908
Trial 5: 259 Steady states. Avg weight=4.2252 Oscil. w avg=3.5578 Ratio=1.1876 Error=0.0061, Rank=889
Trial 6: 312 Steady states. Avg weight=3.7845 Oscil. w avg=3.1697 Ratio=1.194 Error=0.0062, Rank=880
Trial 7: 149 Steady states. Avg weight=3.9373 Oscil. w avg=3.2823 Ratio=1.1996 Error=0.0056, Rank=928
Trial 8: 258 Steady states. Avg weight=4.0292 Oscil. w avg=3.6077 Ratio=1.1168 Error=0.0065, Rank=911
Trial 9: 234 Steady states. Avg weight=3.4593 Oscil. w avg=3.1832 Ratio=1.0867 Error=0.

## Random Digraph (Remove 0%)

In [11]:
rep_ss_ratio("random_digraph", topo_p=4.0, remove_p=0.0)

Trial 0: 1336 Steady states. Avg weight=1.1529 Oscil. w avg=7.2937 Ratio=0.1581 Error=0.0084, Rank=933
Trial 1: 1421 Steady states. Avg weight=1.1329 Oscil. w avg=7.8182 Ratio=0.1449 Error=0.008, Rank=902
Trial 2: 1420 Steady states. Avg weight=1.2723 Oscil. w avg=8.2308 Ratio=0.1546 Error=0.0081, Rank=915
Trial 3: 1073 Steady states. Avg weight=0.9425 Oscil. w avg=6.4796 Ratio=0.1455 Error=0.0084, Rank=936
Trial 4: 875 Steady states. Avg weight=1.1656 Oscil. w avg=6.1836 Ratio=0.1885 Error=0.0079, Rank=989
Trial 5: 1673 Steady states. Avg weight=1.0325 Oscil. w avg=9.8005 Ratio=0.1054 Error=0.0076, Rank=887
Trial 6: 1484 Steady states. Avg weight=1.2957 Oscil. w avg=8.0266 Ratio=0.1614 Error=0.0085, Rank=868
Trial 7: 1436 Steady states. Avg weight=1.1096 Oscil. w avg=7.8542 Ratio=0.1413 Error=0.0087, Rank=912
Trial 8: 1423 Steady states. Avg weight=0.9795 Oscil. w avg=7.5682 Ratio=0.1294 Error=0.0079, Rank=887
Trial 9: 1265 Steady states. Avg weight=1.0693 Oscil. w avg=6.6248 Ratio=0.

## Random Digraph (Remove 80%)

In [33]:
rep_ss_ratio("random_digraph", topo_p=4.0, remove_p=0.8)

Trial 0: 0 Steady states. Avg weight=nan Oscil. w avg=5.6154 Ratio=nan Error=0.0078, Rank=1154
Trial 1: 0 Steady states. Avg weight=nan Oscil. w avg=5.6662 Ratio=nan Error=0.0076, Rank=1140
Trial 2: 1 Steady states. Avg weight=9.6674 Oscil. w avg=6.5225 Ratio=1.4822 Error=0.0088, Rank=1167
Trial 3: 0 Steady states. Avg weight=nan Oscil. w avg=5.4919 Ratio=nan Error=0.0073, Rank=1166


/nas/longleaf/home/djpassey/ReservoirSpecialization/rescomp/ResComp.py:166: UserWarning: Spectral radius of reservoir is close to zero. Edge weights will not be scaled
  warn("Spectral radius of reservoir is close to zero. Edge weights will not be scaled")


Trial 4: 17 Steady states. Avg weight=4.2651 Oscil. w avg=4.5928 Ratio=0.9286 Error=0.0056, Rank=1303
Trial 5: 0 Steady states. Avg weight=nan Oscil. w avg=6.0727 Ratio=nan Error=0.0085, Rank=1152
Trial 6: 0 Steady states. Avg weight=nan Oscil. w avg=5.4481 Ratio=nan Error=0.0075, Rank=1154


/nas/longleaf/home/djpassey/ReservoirSpecialization/rescomp/ResComp.py:166: UserWarning: Spectral radius of reservoir is close to zero. Edge weights will not be scaled
  warn("Spectral radius of reservoir is close to zero. Edge weights will not be scaled")


Trial 7: 19 Steady states. Avg weight=1.6412 Oscil. w avg=4.4715 Ratio=0.367 Error=0.0054, Rank=1254
Trial 8: 0 Steady states. Avg weight=nan Oscil. w avg=5.9195 Ratio=nan Error=0.0081, Rank=1161
Trial 9: 1 Steady states. Avg weight=6.8824 Oscil. w avg=6.1702 Ratio=1.1154 Error=0.0077, Rank=1163
Average: %ss=0.0015 ss_weight=nan os_weight=5.5971 ratio=nan, err=0.007431, rank=1181.4


## Erdos (Remove 0%)

In [13]:
rep_ss_ratio("erdos", topo_p=4.0, remove_p=0.0)

Trial 0: 932 Steady states. Avg weight=1.3349 Oscil. w avg=6.8967 Ratio=0.1936 Error=0.0074, Rank=1261
Trial 1: 932 Steady states. Avg weight=1.0177 Oscil. w avg=4.0548 Ratio=0.251 Error=0.0041, Rank=1633
Trial 2: 1010 Steady states. Avg weight=1.2885 Oscil. w avg=7.5459 Ratio=0.1708 Error=0.0088, Rank=1208
Trial 3: 618 Steady states. Avg weight=0.7879 Oscil. w avg=4.5394 Ratio=0.1736 Error=0.005, Rank=1790
Trial 4: 941 Steady states. Avg weight=1.1409 Oscil. w avg=6.1769 Ratio=0.1847 Error=0.0073, Rank=1246
Trial 5: 721 Steady states. Avg weight=0.8734 Oscil. w avg=4.8892 Ratio=0.1786 Error=0.0052, Rank=1685
Trial 6: 814 Steady states. Avg weight=1.0855 Oscil. w avg=5.5628 Ratio=0.1951 Error=0.0055, Rank=1575
Trial 7: 944 Steady states. Avg weight=1.0222 Oscil. w avg=4.1982 Ratio=0.2435 Error=0.0042, Rank=1606
Trial 8: 809 Steady states. Avg weight=0.8272 Oscil. w avg=5.9588 Ratio=0.1388 Error=0.0069, Rank=1454
Trial 9: 810 Steady states. Avg weight=1.1511 Oscil. w avg=5.8297 Ratio=0.

## Erdos (Remove 90%)

In [14]:
rep_ss_ratio("erdos", topo_p=4.0, remove_p=0.9)

Trial 0: 245 Steady states. Avg weight=3.3124 Oscil. w avg=3.6209 Ratio=0.9148 Error=0.006, Rank=953
Trial 1: 407 Steady states. Avg weight=3.4257 Oscil. w avg=3.5161 Ratio=0.9743 Error=0.0062, Rank=888
Trial 2: 396 Steady states. Avg weight=2.7847 Oscil. w avg=3.3525 Ratio=0.8306 Error=0.0054, Rank=909
Trial 3: 288 Steady states. Avg weight=2.9414 Oscil. w avg=3.3493 Ratio=0.8782 Error=0.0053, Rank=937
Trial 4: 347 Steady states. Avg weight=3.0562 Oscil. w avg=3.6098 Ratio=0.8466 Error=0.0062, Rank=930
Trial 5: 307 Steady states. Avg weight=3.0814 Oscil. w avg=3.596 Ratio=0.8569 Error=0.006, Rank=927
Trial 6: 307 Steady states. Avg weight=3.6288 Oscil. w avg=3.654 Ratio=0.9931 Error=0.0058, Rank=938
Trial 7: 349 Steady states. Avg weight=2.9133 Oscil. w avg=3.6323 Ratio=0.8021 Error=0.0062, Rank=909
Trial 8: 306 Steady states. Avg weight=3.326 Oscil. w avg=3.7198 Ratio=0.8941 Error=0.0059, Rank=929
Trial 9: 424 Steady states. Avg weight=3.6064 Oscil. w avg=3.5479 Ratio=1.0165 Error=0.

# Decrease Spectral Radius

In [15]:
RES_PARAMS["spect_rad"] = 0.5

## Watts (Remove 0%)

In [28]:
rep_ss_ratio("watts4", topo_p=0.1, remove_p=0.0)

Trial 0: 0 Steady states. Avg weight=nan Oscil. w avg=8.2608 Ratio=nan Error=0.0102, Rank=1102
Trial 1: 0 Steady states. Avg weight=nan Oscil. w avg=7.9855 Ratio=nan Error=0.01, Rank=1108
Trial 2: 0 Steady states. Avg weight=nan Oscil. w avg=7.9561 Ratio=nan Error=0.0093, Rank=1093
Trial 3: 0 Steady states. Avg weight=nan Oscil. w avg=7.2986 Ratio=nan Error=0.0092, Rank=1104
Trial 4: 0 Steady states. Avg weight=nan Oscil. w avg=8.0501 Ratio=nan Error=0.0102, Rank=1124
Trial 5: 0 Steady states. Avg weight=nan Oscil. w avg=7.931 Ratio=nan Error=0.0095, Rank=1117
Trial 6: 0 Steady states. Avg weight=nan Oscil. w avg=6.9045 Ratio=nan Error=0.009, Rank=1081
Trial 7: 0 Steady states. Avg weight=nan Oscil. w avg=7.3626 Ratio=nan Error=0.0099, Rank=1114
Trial 8: 0 Steady states. Avg weight=nan Oscil. w avg=7.3834 Ratio=nan Error=0.0095, Rank=1090
Trial 9: 0 Steady states. Avg weight=nan Oscil. w avg=7.2717 Ratio=nan Error=0.0092, Rank=1051
Average: %ss=0.0 ss_weight=nan os_weight=7.6404 ratio=

## Watts (Remove 90%)

In [29]:
rep_ss_ratio("watts4", topo_p=0.1, remove_p=0.9)

Trial 0: 0 Steady states. Avg weight=nan Oscil. w avg=6.3801 Ratio=nan Error=0.0106, Rank=900
Trial 1: 0 Steady states. Avg weight=nan Oscil. w avg=6.4087 Ratio=nan Error=0.0103, Rank=925
Trial 2: 0 Steady states. Avg weight=nan Oscil. w avg=5.7408 Ratio=nan Error=0.0095, Rank=948
Trial 3: 0 Steady states. Avg weight=nan Oscil. w avg=6.5682 Ratio=nan Error=0.0103, Rank=928
Trial 4: 0 Steady states. Avg weight=nan Oscil. w avg=5.8021 Ratio=nan Error=0.0101, Rank=909
Trial 5: 0 Steady states. Avg weight=nan Oscil. w avg=6.3685 Ratio=nan Error=0.0102, Rank=921
Trial 6: 0 Steady states. Avg weight=nan Oscil. w avg=5.9618 Ratio=nan Error=0.0096, Rank=977
Trial 7: 0 Steady states. Avg weight=nan Oscil. w avg=6.0053 Ratio=nan Error=0.0101, Rank=924
Trial 8: 0 Steady states. Avg weight=nan Oscil. w avg=6.4616 Ratio=nan Error=0.0107, Rank=894
Trial 9: 0 Steady states. Avg weight=nan Oscil. w avg=6.4978 Ratio=nan Error=0.0104, Rank=927
Average: %ss=0.0 ss_weight=nan os_weight=6.2195 ratio=nan, e

## Geometric (Remove 0%)

In [18]:
rep_ss_ratio("geom", topo_p=4, remove_p=0.0)

Trial 0: 0 Steady states. Avg weight=nan Oscil. w avg=7.6726 Ratio=nan Error=0.011, Rank=905
Trial 1: 0 Steady states. Avg weight=nan Oscil. w avg=7.1964 Ratio=nan Error=0.0107, Rank=943
Trial 2: 0 Steady states. Avg weight=nan Oscil. w avg=7.9696 Ratio=nan Error=0.0115, Rank=987
Trial 3: 0 Steady states. Avg weight=nan Oscil. w avg=7.4163 Ratio=nan Error=0.0113, Rank=936
Trial 4: 0 Steady states. Avg weight=nan Oscil. w avg=7.9324 Ratio=nan Error=0.0115, Rank=980
Trial 5: 0 Steady states. Avg weight=nan Oscil. w avg=7.7633 Ratio=nan Error=0.0112, Rank=986
Trial 6: 0 Steady states. Avg weight=nan Oscil. w avg=8.3107 Ratio=nan Error=0.0116, Rank=984
Trial 7: 0 Steady states. Avg weight=nan Oscil. w avg=8.2815 Ratio=nan Error=0.0115, Rank=994
Trial 8: 0 Steady states. Avg weight=nan Oscil. w avg=8.5828 Ratio=nan Error=0.012, Rank=973
Trial 9: 0 Steady states. Avg weight=nan Oscil. w avg=7.6493 Ratio=nan Error=0.0113, Rank=972
Average: %ss=0.0 ss_weight=nan os_weight=7.8775 ratio=nan, err

## Geometric (Remove 90%)

In [19]:
rep_ss_ratio("geom", topo_p=4.0, remove_p=0.9)

Trial 0: 0 Steady states. Avg weight=nan Oscil. w avg=5.8393 Ratio=nan Error=0.0097, Rank=925
Trial 1: 0 Steady states. Avg weight=nan Oscil. w avg=6.4165 Ratio=nan Error=0.0106, Rank=914
Trial 2: 0 Steady states. Avg weight=nan Oscil. w avg=6.4023 Ratio=nan Error=0.0106, Rank=911
Trial 3: 0 Steady states. Avg weight=nan Oscil. w avg=6.3058 Ratio=nan Error=0.0102, Rank=897
Trial 4: 0 Steady states. Avg weight=nan Oscil. w avg=6.2374 Ratio=nan Error=0.0096, Rank=891
Trial 5: 0 Steady states. Avg weight=nan Oscil. w avg=6.1922 Ratio=nan Error=0.0102, Rank=911
Trial 6: 0 Steady states. Avg weight=nan Oscil. w avg=6.3905 Ratio=nan Error=0.0102, Rank=898
Trial 7: 0 Steady states. Avg weight=nan Oscil. w avg=5.8368 Ratio=nan Error=0.0105, Rank=885
Trial 8: 0 Steady states. Avg weight=nan Oscil. w avg=6.6657 Ratio=nan Error=0.0107, Rank=953
Trial 9: 0 Steady states. Avg weight=nan Oscil. w avg=6.458 Ratio=nan Error=0.0103, Rank=890
Average: %ss=0.0 ss_weight=nan os_weight=6.2744 ratio=nan, er

## Identity (Remove 0%)

In [20]:
rep_ss_ratio("ident", topo_p=None, remove_p=0.0)

Trial 0: 0 Steady states. Avg weight=nan Oscil. w avg=13.8156 Ratio=nan Error=0.0172, Rank=679
Trial 1: 0 Steady states. Avg weight=nan Oscil. w avg=12.4551 Ratio=nan Error=0.0161, Rank=658
Trial 2: 0 Steady states. Avg weight=nan Oscil. w avg=12.914 Ratio=nan Error=0.0175, Rank=675
Trial 3: 0 Steady states. Avg weight=nan Oscil. w avg=14.8698 Ratio=nan Error=0.0189, Rank=692
Trial 4: 0 Steady states. Avg weight=nan Oscil. w avg=13.2298 Ratio=nan Error=0.0177, Rank=688
Trial 5: 0 Steady states. Avg weight=nan Oscil. w avg=12.0731 Ratio=nan Error=0.0164, Rank=670
Trial 6: 0 Steady states. Avg weight=nan Oscil. w avg=13.1108 Ratio=nan Error=0.017, Rank=685
Trial 7: 0 Steady states. Avg weight=nan Oscil. w avg=12.4056 Ratio=nan Error=0.016, Rank=686
Trial 8: 0 Steady states. Avg weight=nan Oscil. w avg=12.305 Ratio=nan Error=0.0161, Rank=676
Trial 9: 0 Steady states. Avg weight=nan Oscil. w avg=13.195 Ratio=nan Error=0.0165, Rank=686
Average: %ss=0.0 ss_weight=nan os_weight=13.0374 ratio=

## Identity (Remove 90%)

In [21]:
rep_ss_ratio("ident", topo_p=None, remove_p=0.9)

Trial 0: 0 Steady states. Avg weight=nan Oscil. w avg=7.4938 Ratio=nan Error=0.0135, Rank=536
Trial 1: 0 Steady states. Avg weight=nan Oscil. w avg=7.3518 Ratio=nan Error=0.014, Rank=531
Trial 2: 0 Steady states. Avg weight=nan Oscil. w avg=6.6718 Ratio=nan Error=0.013, Rank=519
Trial 3: 0 Steady states. Avg weight=nan Oscil. w avg=7.2978 Ratio=nan Error=0.0132, Rank=519
Trial 4: 0 Steady states. Avg weight=nan Oscil. w avg=7.5264 Ratio=nan Error=0.014, Rank=521
Trial 5: 0 Steady states. Avg weight=nan Oscil. w avg=7.0663 Ratio=nan Error=0.0135, Rank=517
Trial 6: 0 Steady states. Avg weight=nan Oscil. w avg=7.6903 Ratio=nan Error=0.0132, Rank=521
Trial 7: 0 Steady states. Avg weight=nan Oscil. w avg=6.8122 Ratio=nan Error=0.0128, Rank=521
Trial 8: 0 Steady states. Avg weight=nan Oscil. w avg=7.3874 Ratio=nan Error=0.0135, Rank=528
Trial 9: 0 Steady states. Avg weight=nan Oscil. w avg=6.3614 Ratio=nan Error=0.0119, Rank=509
Average: %ss=0.0 ss_weight=nan os_weight=7.1659 ratio=nan, err=

## Barabasi (Remove 0%)

In [22]:
rep_ss_ratio("barab2", topo_p=None, remove_p=0.0)

Trial 0: 0 Steady states. Avg weight=nan Oscil. w avg=7.7198 Ratio=nan Error=0.0117, Rank=944
Trial 1: 0 Steady states. Avg weight=nan Oscil. w avg=7.4512 Ratio=nan Error=0.0126, Rank=894
Trial 2: 0 Steady states. Avg weight=nan Oscil. w avg=7.4925 Ratio=nan Error=0.0121, Rank=899
Trial 3: 0 Steady states. Avg weight=nan Oscil. w avg=8.3001 Ratio=nan Error=0.0124, Rank=892
Trial 4: 0 Steady states. Avg weight=nan Oscil. w avg=7.7078 Ratio=nan Error=0.0116, Rank=867
Trial 5: 0 Steady states. Avg weight=nan Oscil. w avg=7.966 Ratio=nan Error=0.0119, Rank=909
Trial 6: 0 Steady states. Avg weight=nan Oscil. w avg=7.0982 Ratio=nan Error=0.0119, Rank=907
Trial 7: 0 Steady states. Avg weight=nan Oscil. w avg=7.8552 Ratio=nan Error=0.013, Rank=925
Trial 8: 0 Steady states. Avg weight=nan Oscil. w avg=7.6801 Ratio=nan Error=0.0118, Rank=912
Trial 9: 0 Steady states. Avg weight=nan Oscil. w avg=7.6861 Ratio=nan Error=0.0124, Rank=860
Average: %ss=0.0 ss_weight=nan os_weight=7.6957 ratio=nan, err

## Barabasi (Remove 90%)

In [23]:
rep_ss_ratio("barab2", topo_p=None, remove_p=0.9)

Trial 0: 0 Steady states. Avg weight=nan Oscil. w avg=6.1338 Ratio=nan Error=0.01, Rank=912
Trial 1: 0 Steady states. Avg weight=nan Oscil. w avg=6.3952 Ratio=nan Error=0.0099, Rank=902
Trial 2: 0 Steady states. Avg weight=nan Oscil. w avg=6.8931 Ratio=nan Error=0.0124, Rank=873
Trial 3: 0 Steady states. Avg weight=nan Oscil. w avg=6.7213 Ratio=nan Error=0.0111, Rank=905
Trial 4: 0 Steady states. Avg weight=nan Oscil. w avg=6.3103 Ratio=nan Error=0.0103, Rank=879
Trial 5: 0 Steady states. Avg weight=nan Oscil. w avg=5.9412 Ratio=nan Error=0.0099, Rank=890
Trial 6: 0 Steady states. Avg weight=nan Oscil. w avg=6.7727 Ratio=nan Error=0.0111, Rank=890
Trial 7: 1 Steady states. Avg weight=3.954 Oscil. w avg=6.6571 Ratio=0.594 Error=0.0104, Rank=889
Trial 8: 0 Steady states. Avg weight=nan Oscil. w avg=6.0948 Ratio=nan Error=0.0097, Rank=894
Trial 9: 0 Steady states. Avg weight=nan Oscil. w avg=6.7743 Ratio=nan Error=0.0112, Rank=901
Average: %ss=0.0 ss_weight=nan os_weight=6.4694 ratio=nan,

## Random Digraph (Remove 0%)

In [24]:
rep_ss_ratio("random_digraph", topo_p=4.0, remove_p=0.0)

Trial 0: 0 Steady states. Avg weight=nan Oscil. w avg=7.8613 Ratio=nan Error=0.0095, Rank=1094
Trial 1: 0 Steady states. Avg weight=nan Oscil. w avg=6.9339 Ratio=nan Error=0.0097, Rank=1109
Trial 2: 0 Steady states. Avg weight=nan Oscil. w avg=7.1995 Ratio=nan Error=0.0088, Rank=1063
Trial 3: 0 Steady states. Avg weight=nan Oscil. w avg=7.1586 Ratio=nan Error=0.0096, Rank=1071
Trial 4: 0 Steady states. Avg weight=nan Oscil. w avg=7.2901 Ratio=nan Error=0.0097, Rank=1102
Trial 5: 0 Steady states. Avg weight=nan Oscil. w avg=7.4336 Ratio=nan Error=0.0096, Rank=1077
Trial 6: 0 Steady states. Avg weight=nan Oscil. w avg=7.3825 Ratio=nan Error=0.0099, Rank=1112
Trial 7: 0 Steady states. Avg weight=nan Oscil. w avg=7.2124 Ratio=nan Error=0.0096, Rank=1104
Trial 8: 0 Steady states. Avg weight=nan Oscil. w avg=7.1807 Ratio=nan Error=0.0095, Rank=1063
Trial 9: 0 Steady states. Avg weight=nan Oscil. w avg=6.8175 Ratio=nan Error=0.0091, Rank=1064
Average: %ss=0.0 ss_weight=nan os_weight=7.247 rat

## Random Digraph (Remove 80%)

In [32]:
rep_ss_ratio("random_digraph", topo_p=4.0, remove_p=0.8)

Trial 0: 0 Steady states. Avg weight=nan Oscil. w avg=5.251 Ratio=nan Error=0.0071, Rank=1159
Trial 1: 0 Steady states. Avg weight=nan Oscil. w avg=5.6 Ratio=nan Error=0.0078, Rank=1165
Trial 2: 0 Steady states. Avg weight=nan Oscil. w avg=5.859 Ratio=nan Error=0.0076, Rank=1117
Trial 3: 0 Steady states. Avg weight=nan Oscil. w avg=5.812 Ratio=nan Error=0.008, Rank=1175
Trial 4: 1 Steady states. Avg weight=3.8972 Oscil. w avg=5.4381 Ratio=0.7166 Error=0.0071, Rank=1165
Trial 5: 0 Steady states. Avg weight=nan Oscil. w avg=5.864 Ratio=nan Error=0.0077, Rank=1159
Trial 6: 0 Steady states. Avg weight=nan Oscil. w avg=5.4611 Ratio=nan Error=0.0074, Rank=1147


/nas/longleaf/home/djpassey/ReservoirSpecialization/rescomp/ResComp.py:166: UserWarning: Spectral radius of reservoir is close to zero. Edge weights will not be scaled
  warn("Spectral radius of reservoir is close to zero. Edge weights will not be scaled")


Trial 7: 15 Steady states. Avg weight=3.9068 Oscil. w avg=4.6957 Ratio=0.832 Error=0.0055, Rank=1281
Trial 8: 1 Steady states. Avg weight=6.1462 Oscil. w avg=5.645 Ratio=1.0888 Error=0.0078, Rank=1109
Trial 9: 0 Steady states. Avg weight=nan Oscil. w avg=5.4638 Ratio=nan Error=0.0073, Rank=1146
Average: %ss=0.0007 ss_weight=nan os_weight=5.509 ratio=nan, err=0.007347, rank=1162.3


## Erdos (Remove 0%)

In [26]:
rep_ss_ratio("erdos", topo_p=4.0, remove_p=0.0)

Trial 0: 0 Steady states. Avg weight=nan Oscil. w avg=7.3288 Ratio=nan Error=0.0102, Rank=1048
Trial 1: 0 Steady states. Avg weight=nan Oscil. w avg=7.5063 Ratio=nan Error=0.0104, Rank=1027
Trial 2: 0 Steady states. Avg weight=nan Oscil. w avg=7.7667 Ratio=nan Error=0.0103, Rank=1005
Trial 3: 0 Steady states. Avg weight=nan Oscil. w avg=7.3585 Ratio=nan Error=0.0099, Rank=1024
Trial 4: 0 Steady states. Avg weight=nan Oscil. w avg=7.6347 Ratio=nan Error=0.0096, Rank=1040
Trial 5: 0 Steady states. Avg weight=nan Oscil. w avg=7.7343 Ratio=nan Error=0.0107, Rank=1049
Trial 6: 0 Steady states. Avg weight=nan Oscil. w avg=7.6257 Ratio=nan Error=0.0097, Rank=1014
Trial 7: 0 Steady states. Avg weight=nan Oscil. w avg=7.6544 Ratio=nan Error=0.0097, Rank=1049
Trial 8: 0 Steady states. Avg weight=nan Oscil. w avg=7.8395 Ratio=nan Error=0.0098, Rank=1051
Trial 9: 0 Steady states. Avg weight=nan Oscil. w avg=8.1328 Ratio=nan Error=0.0106, Rank=1034
Average: %ss=0.0 ss_weight=nan os_weight=7.6582 ra

## Erdos (Remove 90%)

In [27]:
rep_ss_ratio("erdos", topo_p=4.0, remove_p=0.9)

Trial 0: 0 Steady states. Avg weight=nan Oscil. w avg=6.6349 Ratio=nan Error=0.0104, Rank=890
Trial 1: 0 Steady states. Avg weight=nan Oscil. w avg=6.024 Ratio=nan Error=0.0094, Rank=929
Trial 2: 0 Steady states. Avg weight=nan Oscil. w avg=5.2782 Ratio=nan Error=0.0091, Rank=952
Trial 3: 0 Steady states. Avg weight=nan Oscil. w avg=6.7368 Ratio=nan Error=0.0108, Rank=936
Trial 4: 0 Steady states. Avg weight=nan Oscil. w avg=6.658 Ratio=nan Error=0.0113, Rank=890
Trial 5: 0 Steady states. Avg weight=nan Oscil. w avg=7.3933 Ratio=nan Error=0.0113, Rank=907
Trial 6: 0 Steady states. Avg weight=nan Oscil. w avg=6.4215 Ratio=nan Error=0.0108, Rank=908
Trial 7: 0 Steady states. Avg weight=nan Oscil. w avg=6.3622 Ratio=nan Error=0.01, Rank=908
Trial 8: 0 Steady states. Avg weight=nan Oscil. w avg=6.1228 Ratio=nan Error=0.0103, Rank=922
Trial 9: 0 Steady states. Avg weight=nan Oscil. w avg=6.029 Ratio=nan Error=0.0102, Rank=930
Average: %ss=0.0 ss_weight=nan os_weight=6.3661 ratio=nan, err=0.